# Welcome

We are going to be gathering weather data from the past year and throwing it into an SQLite database.
I realized on v1 that I was just throwing away data. Not good.

In [26]:
import requests
import json
from datetime import datetime

In [3]:
key_path = 'C:/Users/asalm/My Drive (asalmon2001@gmail.com)/Github Repos/Secrets'
key_file = open(key_path + '/weatherapi.txt', 'r')
api_key = key_file.read()

Lets check what historical data we can get

In [4]:
params = { 'key':api_key , 'q':'Calgary', 'dt':'2023-09-14' }
sample_response = requests.get('http://api.weatherapi.com/v1/history.json', params=params)
print(sample_response.status_code)

200


In [5]:
print(json.dumps(obj=sample_response.json(), indent=2))

{
  "location": {
    "name": "Calgary",
    "region": "Alberta",
    "country": "Canada",
    "lat": 51.08,
    "lon": -114.08,
    "tz_id": "America/Edmonton",
    "localtime_epoch": 1695325359,
    "localtime": "2023-09-21 13:42"
  },
  "forecast": {
    "forecastday": [
      {
        "date": "2023-09-14",
        "date_epoch": 1694649600,
        "day": {
          "maxtemp_c": 22.7,
          "maxtemp_f": 72.9,
          "mintemp_c": 9.0,
          "mintemp_f": 48.2,
          "avgtemp_c": 14.6,
          "avgtemp_f": 58.3,
          "maxwind_mph": 6.7,
          "maxwind_kph": 10.8,
          "totalprecip_mm": 0.0,
          "totalprecip_in": 0.0,
          "avgvis_km": 9.7,
          "avgvis_miles": 6.0,
          "avghumidity": 56.0,
          "condition": {
            "text": "Sunny",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/113.png",
            "code": 1000
          },
          "uv": 6.0
        },
        "astro": {
          "sunrise": "07:10 AM",
 

Lets think about what tables we want.
1. Forecast_hourly_data
   Columns: hour_id (ie. 23091403), *all the columns in each hour (minus condition: that we will just keep the code)
3. Forecast_day_data
4. Astro_data
5. 

In [11]:
import sqlite3 as sl
connection = sl.connect('weather_v2_database.db')
print(connection.total_changes)

0


In [97]:
def conversion(element) :
    return '\'' + str(element) + '\''

In [137]:
# Produce all the column names from the json
sample_hourly = sample_response.json()['forecast']['forecastday'][0]['hour']
column_names = ['hour_id']
key_value_list = list(sample_hourly[0].items())
for key, value in key_value_list[1:4] :
    column_names.append(key)
for key, value in key_value_list[6:] :
    column_names.append(key)
column_names.append('condition_text')
column_names.append('condition_code')

In [145]:
def deleteHourlyWeatherTable() :
    try:
        cursor.execute('DROP TABLE hourly_weather')
    except:
        print('TABLE hourly_weather DOES NOT EXIST')

In [146]:
# Create hourly_weather table
cursor = connection.cursor()
command = command = 'CREATE TABLE hourly_weather ( ' + ', '.join(list(map(conversion, column_names))) + ')'

deleteHourlyWeatherTable()
cursor.execute(command)
printHourlyWeather()

TABLE hourly_weather DOES NOT EXIST
[]


In [95]:
# This is how I will format the hour_id
def getHourIdandTime(hour) :
    my_datetime = datetime.fromisoformat(hour['time'])
    [hour_id, time] = my_datetime.strftime('%y%m%d%H-%H:%M').split('-')
    return [hour_id, time]

In [96]:
# Printing the database
def printHourlyWeather() :
    rows = cursor.execute('SELECT * FROM hourly_weather').fetchall()
    print(rows)

In [106]:
def insertHourlyData(hour) :
    insertion_values = getHourIdandTime(hour)
    key_value_list = 
    
    
    
    # temp_c = hour['temp_c']
    # temp_f = hour['temp_f']
    # is_day = hour['is_day']
    # condition_desc = hour['condition']['text']
    # condition_code = hour['condition']['code']
    # insertion_list = list(map(conversion, [hour_id, time, temp_c, temp_f, is_day, condition_desc, condition_code]))
    # command = 'INSERT INTO hourly_weather VALUES (' + ', '.join(insertion_list) + ')'
    # cursor.execute(command)
    # #print(command)

In [147]:
sample_hourly = sample_response.json()['forecast']['forecastday'][0]['hour']
#for hour in sample_hourly :
#    insertHourlyData(hour)
print(sample_hourly[0])

{'time_epoch': 1694671200, 'time': '2023-09-14 00:00', 'temp_c': 13.4, 'temp_f': 56.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 351, 'wind_dir': 'N', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 54, 'cloud': 56, 'feelslike_c': 12.6, 'feelslike_f': 54.7, 'windchill_c': 12.6, 'windchill_f': 54.7, 'heatindex_c': 13.4, 'heatindex_f': 56.1, 'dewpoint_c': 4.4, 'dewpoint_f': 39.9, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 10.7, 'gust_kph': 17.3, 'uv': 1.0}


We now have a way to insert an entire day of hourly data into the hourly_weather table